# Neural Network
- Neural network can be constructed using the torch.nn package
- An `nn.Module` contains layers, and a method forward(input) that returns the output

## Simple Feed-forward Network
- In a simple feed-forward network that classifies digit images:
    + Takes the input
    + Feeds it through several layers one after other
    + Gives the output

## Training procedure
- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss
- Propagate gradients back into the network's parameters
- Update the weigths of the network, typically using a simple update rule
    + weight = weight - learning_rate * gradient

# Define the network

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # An affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling per a 2 (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)     # Flatten all dimensioo except the batch dimension
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Define `forward` and `backward` Function
- You have to define the `forward` function
- The `backward` function (where gradients are computed) is automatically defined for you using autograd
- You can use any of the Tensor operatio in the `forward` function
- The learnable parameters of a model are returned by `net.parameters()`

In [33]:
params = list(net.parameters())
print(len(params))
print(params[0].size())     #conv1's .weight

10
torch.Size([6, 1, 5, 5])


## Input
- Try a random 32x32 input
- Note: expect input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32

In [34]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0814, -0.0772,  0.0230,  0.1087, -0.0084, -0.0196,  0.0867,  0.1305,
         -0.0219, -0.0027]], grad_fn=<AddmmBackward0>)


## Backprops
- Zero the gradient buffers of all parameters and backprops with random gradients

In [35]:
net.zero_grad()
out.backward(torch.randn(1, 10))

## Note
- `torch.nn` only support mini-batches. The entire `torch.nn` package only supports inputs that are mini-batch of smaples, and not a single sample
- For example, `nn.Conv2d` will take in a 4D Tensor of `nSamples x nChannels x Height x Width`
- If you have single sample, just use `input.unsqueeze(0)` to add a fake batch dimension

## Recap
- `torch.Tensor` = a multi-dimensional array with support for autograd operations like backward()
    + Also hold the gradient w.r.t the tensor
- `nn.Module` = Neural Network module
    + Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
- `nn.Parameters` = A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a `Module`
- `autograd.Function` = Implements forward and backward definitions of an utograd operation
    + Every `Tensor` operation creates at least a single `Function` node that connects to functions that created a `Tensor` and encodes its history

# Loss Function
- A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target
- There are several different `loss functions` under the nn package

## Mean-squared Error 
- A simple loss is `nn.MSELoss` which computes the mean-squared error between the output and the target

In [36]:
output = net(input)
target = torch.randn(10)        # a dummy target, for example
target = target.view(1, -1)     # Make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6203, grad_fn=<MseLossBackward0>)


## Follow loss in the backward direction
- If you follow loss in the backward direction, using its `.grad_fn` attribute, you will see a graph of computations
    + input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
            -> flatten -> linear -> relu -> linear -> relu -> linear
            -> MSELoss
            -> loss
- When we call `loss.backward()`, the whole graph is differentiated w.r.t the neural net parameters, and all the Tensors in the graph that have `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient

In [37]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## Backprop
- To backpropagate the error all we have to do is to `loss.backward()`
- You need to clear existing gradients through, else gradients will be accumulated to existing gradients

In [38]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0006,  0.0067,  0.0026, -0.0148, -0.0014,  0.0042])


## Extra documents
- The neural network package contains various modules and loss functions that form the building blocks of deep neural networks. A full list with documentation is here https://pytorch.org/docs/stable/nn.html

# Update the weights
- The simplest update rule used in practice is the Stochastic Gradient Descent (SGD)
    + `weight = weight - learning_rate * gradient`

In [39]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

## Use more update rules than one
- However, as you use neural network, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
- To enable this, we built a small package: `torch.optim` that implements all these methods

In [40]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update